# Save IGRINS observation times to a machine readable log

We'll need this to register the TESS observations.

### Get the IGRINS epoch times.

In [1]:
import glob

In [2]:
from astropy.io import fits

In [3]:
reduced_fns = glob.glob('../../data/IGRINS/GS-2021A-Q-311/*/reduced/SDCH*spec_a0v.fits')

In [4]:
raw_fns = glob.glob('../../data/IGRINS/GS-2021A-Q-311/*/SDCH*.fits')

In [5]:
len(reduced_fns)

24

In [6]:
len(raw_fns)

139

In [7]:
hdu = fits.open(raw_fns[23])[0]
hdr = hdu.header

In [8]:
keep_keys = ['OBJECT', 'EXPTIME', 'FRMTYPE', 'OBJTYPE', 'JD-OBS']

In [9]:
import pandas as pd

In [10]:
df_raw = pd.DataFrame()

for i, fn in enumerate(raw_fns):
    try:
        hdu = fits.open(fn)[0]
        hdr = hdu.header
        out_dict = {key:hdr[key] for key in keep_keys}
        out_dict['night_dir'] = fn[33:33+8]
        df = pd.DataFrame(out_dict, index=[i])
        df_raw = df_raw.append(df)
    except:
        print(f"{fn} had a problem")

In [11]:
df_log = df_raw.sort_values('JD-OBS').reset_index(drop=True)

Make a quick summary of how many spectra were acquired on each night:

In [12]:
from astropy.time import Time

In [13]:
df_log['BTJD'] = df_log['JD-OBS']- 2457000.0

In [14]:
df_out = df_log.groupby(['night_dir', 'OBJECT', 'EXPTIME']).BTJD.mean().round(decimals=4).to_frame()
df_out

BTJD
night_dir OBJECT                  EXPTIME           
20210202  UCAC2 7201471           30.0     2248.6363
20210209  TYC 8534-1243-1         70.0     2255.5876
20210210  HD 49855                10.0     2256.5614
          TYC 8534-1243-1         50.0     2256.5409
          UCAC2 7201471           20.0     2256.5551
20210213  HD 49855                30.0     2259.5557
          TYC 8534-1243-1         120.0    2259.5275
          UCAC2 7201471           60.0     2259.5487
20210215  HD 49855                30.0     2261.5832
          TYC 8534-1243-1         120.0    2261.5554
          UCAC2 7201471           30.0     2261.5737
20210216  HD 49855                4.0      2262.6009
          TYC 8534-1243-1         80.0     2262.5276
          UCAC2 7201471           30.0     2262.5426
20210217  HD 49855                20.0     2263.5394
20210221  UCAC2 7201471           30.0     2267.5131
20210310  V* V479 Car             16.0     2284.5063
20210313  V* V1249 Cen            30.0     2287.7163
          V* V479 Car             30.0     2287.6958
20210314  2MASS J12271665-6239142 30.0     2288.5819
          CD-62 657               30.0     2288.5974
          V* V1249 Cen            30.0     2288.5647
          V* V479 Car             30.0     2288.5460
20210316  V* V479 Car             30.0     2290.6759
20210317  V* V479 Car             30.0     2291.6196

Neat! Let's save these so we can read them in later.

In [15]:
df_out.to_csv('../../data/IGRINS/2021A_metadata_log.csv')

In [16]:
! head ../../data/IGRINS/2021A_metadata_log.csv

night_dir,OBJECT,EXPTIME,BTJD
20210202,UCAC2 7201471,30.0,2248.6363
20210209,TYC 8534-1243-1,70.0,2255.5876
20210210,HD 49855,10.0,2256.5614
20210210,TYC 8534-1243-1,50.0,2256.5409
20210210,UCAC2 7201471,20.0,2256.5551
20210213,HD 49855,30.0,2259.5557
20210213,TYC 8534-1243-1,120.0,2259.5275
20210213,UCAC2 7201471,60.0,2259.5487
20210215,HD 49855,30.0,2261.5832


Great!  Now we can take those dates and overlay them on TESS lightcurves.